<a href="https://colab.research.google.com/github/NicholasW179/NLP-with-Python/blob/master/testng_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install texthero

     |████████████████████████████████| 1.4MB 5.9MB/s 
     |████████████████████████████████| 245kB 16.9MB/s 
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434675 sha256=4b020a10b2b7697d070870e7b05ba87f9a1a66b7427c3a8c8b9dee5ed8d6558e
  Stored in directory: /root/.cache/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
import os

import pandas as pd
import numpy as np
import tensorflow_hub as hub
import gensim
import tensorflow as tf
#import transformers
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
from tensorflow.keras.layers import Dense
tf.config.list_physical_devices("GPU")
tf.test.is_built_with_cuda()
tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None)
from sklearn.cluster import KMeans
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import re
import texthero as hero
from texthero import preprocessing
import multiprocessing
cores = multiprocessing.cpu_count()
df = pd.read_csv('testing_data.csv')
df.dropna(subset=['abstract'], inplace=True)


empirical_journals = ['experimental economics', 'journal of applied economics',
                      'empirical economics', 'american economic journal: applied economics',
                      'applied economics perspectives and policy', 
                      'applied economics and finance', 'applied economics',
                      ]

theoretical_journals = ['journal of economics theory', 'american economic journal: microeconomics',
                        'economic theory', 'theoretical economics']



# import bs4 for cleaning
'''
from bs4 import BeautifulSoup
def cleanText(text):
    text = BeautifulSoup(text, "lxml").text
    text = re.sub(r'\|\|\|', r' ', text) 
    text = re.sub(r'http\S+', r'<URL>', text)
    text = text.lower()
    text = text.replace('x', '')
    return text
'''


custom_pipe = [preprocessing.fillna,
               #preprocessing.remove_brackets,
               preprocessing.lowercase,
               preprocessing.remove_whitespace,
               preprocessing.remove_html_tags,
               preprocessing.remove_diacritics,
               ]

df['abstract'] = hero.clean(df['abstract'], custom_pipe)
df['abstract'] = [n.replace('{','') for n in df['abstract']]
df['abstract'] = [n.replace('}','') for n in df['abstract']]
df['abstract'] = [n.replace('(','') for n in df['abstract']]
df['abstract'] = [n.replace(')','') for n in df['abstract']]



econometrics_journals = ['journal of econometrics', 'econometric theory']


df['journal'] = df.journal.str.lower()
empirical = df.journal.isin(empirical_journals)
theoretical = df.journal.isin(theoretical_journals)
econometric = df.journal.isin(econometrics_journals)



# Get training and test data, label target variables

df['theoretical'] = df.journal.isin(theoretical_journals)
df['empirical'] = df.journal.isin(empirical_journals)
labels = df[['theoretical', 'empirical']]*1
y = labels.idxmax(axis =1)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
train, test,y_train, y_test = train_test_split(df,y, test_size = 0.1, random_state =0)


# Prepare for doc2vec

tagged_documents = [TaggedDocument(t.split(' '), [i])
                    for i, t in enumerate(df.abstract)]

# Instantiate model

model = Doc2Vec(vector_size = 100, window = 5, alpha = .025, 
                min_alpha = .00025, min_count =1, dm =1,
                workers = cores)



# Tag training & Test documents

tagged_train = [TaggedDocument(t.split(' '), [i]) for i,t in
                enumerate(train.abstract)]


tagged_test = [TaggedDocument(t.split(' '),[i]) for i, t in enumerate(test.abstract)]

# Get vocab and build doc2vec models on training set

vocab = model.build_vocab(tagged_train)

model.train(tagged_train, total_examples = model.corpus_count,epochs = 250)
model.save('100_dim_doc2vec_model')



doctags = model.docvecs.vectors_docs

x_train = np.array([model.docvecs[i] for i in range(len(tagged_train))])
x_test = np.array([model.infer_vector(tagged_test[i][0])
                   for i in range(len(tagged_test))])

kmeans = KMeans(init =  'k-means++',n_clusters = 2, max_iter = 700)
kmeans.fit(x_train, y_train)
klabels = kmeans.labels_
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics import accuracy_score
score = adjusted_rand_score(y_train, klabels)



preds = kmeans.predict(x_test)
score_pred = adjusted_rand_score(y_test, preds)
accuracy_pred = accuracy_score(y_test, preds)


# Try again with PCA
pca = PCA(n_components= 2)
pca_vecs = pca.fit_transform(x_train)
kmeans.fit(pca_vecs)
klabels = kmeans.labels_
score2 = adjusted_rand_score(y_train, klabels)
accuracy2 = accuracy_score(y_train, klabels)

preds2 = kmeans.predict(pca.fit_transform(x_test))
pscore_pred = adjusted_rand_score(y_test, preds2)
accuracy_pred2 = accuracy_score(y_test, preds2)



# Look at it with entire dataset

model = Doc2Vec(vector_size = 100, window = 5, alpha = .025, 
                min_alpha = .00025, min_count =1, dm =1,
                workers = cores)





# do the same for 3oo dimensions






model = Doc2Vec(vector_size = 300, window = 5, alpha = .025, 
                min_alpha = 0.00025, min_count =1, dm =1, workers = cores)











# Prepare with universal sentence encoder
train, test = train_test_split(df, test_size = 0.01)
embed = hub.load('https://tfhub.dev/google/universal-sentence-encoder-large/5')
embeddings = embed(df['abstract'])
usage = np.array(embeddings).tolist()
kmeans.fit(usage, y = y)


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


ModuleNotFoundError: ignored